In [1]:
from polygon import RESTClient
from typing import cast
from urllib3 import HTTPResponse
import json
import pandas as pd
import datetime
import pytz

API_KEY = "I0D0ldkssJgACZm3fcs9DlZAX5osmgZp"

In [2]:
client = RESTClient(api_key=API_KEY)

In [3]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
last_year = (datetime.datetime.now() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

In [4]:
utc = pytz.utc
est = pytz.timezone("US/Eastern")

ticker = "AAPL"
aggs = []

for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="day", from_=last_year, to=today, limit=50000):
    aggs.append(a)


In [5]:
df = pd.DataFrame(aggs)
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)  # Convert to UTC first
df["timestamp"] = df["timestamp"].dt.tz_convert(est)  # Convert to EST

In [10]:
aggs = cast(
    HTTPResponse,
    client.get_aggs(ticker=ticker, multiplier=1, timespan="day", from_=last_year, to=today, limit=50000,raw=True),
)

In [15]:
data_str = aggs.data.decode("utf-8")
data = json.loads(data_str)
df = pd.DataFrame(data["results"])

In [18]:
data_str = aggs.data.decode("utf-8")
data = json.loads(data_str)
df = pd.DataFrame(data["results"])

df.sort_values(by="t", ascending=False, inplace=True)
df["Date"] = pd.to_datetime(df["t"], unit="ms", utc=True).dt.tz_convert("America/New_York")

# Format to display date in NYC timezone
df["Date"] = df["Date"].dt.strftime("%d-%m-%Y %H:%M:%S %Z")

# Sort values by Date in descending order
#df.sort_values(by="Date", ascending=False, inplace=True)

# Rename columns
df.rename(columns={
    "o": "Open",
    "c": "Close",
    "h": "High",
    "l": "Low",
    "v": "Volume",
    "vw": "VWAP",
    "n": "Trades"
}, inplace=True)

# Drop original timestamp column
df.drop(columns=["t"], inplace=True)

In [20]:
df.set_index("Date", inplace=True)